In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import os

In [ ]:

url = "https://www.melon.com/chart/index.htm"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"}

html = requests.get(url, headers=headers)

# 응답을 BeautifulSoup 객체로 파싱합니다.
soup = BeautifulSoup(html.content, "html.parser")
music_num_all = []
# 첫 번째 tr 요소를 찾습니다.
music_num50 = soup.find_all("tr", {"class": "lst50", "id": "lst50"})
for i in music_num50:
# data-song-no 속성 값을 가져옵니다.
    data_song_no = i.get("data-song-no")
    music_num_all.append(data_song_no)
    print(data_song_no)

music_num100 = soup.find_all("tr", {"class":"lst100","id":"lst100"})

for i in music_num100:
    data_song_no = i.get("data-song-no")
    music_num_all.append(data_song_no)
    print(data_song_no)




In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import re

# music_num_all 리스트 정의가 필요합니다.
# music_num_all = [ ... ]

for i in music_num_all:
    url = f"https://www.melon.com/song/detail.htm?songId="+i
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"
    }
    html = requests.get(url, headers=headers)
    soup = BeautifulSoup(html.content, 'html.parser')

    music_name = soup.find('div', class_='song_name').text
    text = re.sub(r"\s+", "", music_name)
    music_text = re.sub(r"곡명", "", text)

    music_artist = soup.find('div',class_='artist').text
    artist_text = re.sub(r"\s+", "", music_artist)

    music_lyrics = soup.find('div',class_='lyric').get_text("\n")
    lines = music_lyrics.split('\n')
    cleaned_lyrics = '\n'.join(line.lstrip() for line in lines if line.strip())

    folder_path = "lyrics"

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # 중복 파일 이름 체크
    file_path = os.path.join(folder_path, f'{artist_text}-{music_text}.txt')
    if os.path.exists(file_path):
        print(f"파일 '{file_path}' 이미 존재합니다. 중복된 파일을 저장하지 않습니다.")
        continue

    # 파일을 쓰기 모드로 열고 텍스트를 파일에 작성
    with open(file_path, 'w') as file:
        file.write(cleaned_lyrics)
        print(f"파일 '{file_path}' 저장 완료")

In [ ]:
import os
import pandas as pd

# set the folder path where the lyrics files are stored
folder_path = "/Users/dryoon04/Documents/GitHub/schoolproject/vectorstore/lyrics"

# create an empty list to store the data
data = []

# loop through all the files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        # extract the artist and title from the filename
        artist, title = filename[:-4].rsplit("-", 1)

        # read the lyrics from the file
        with open(os.path.join(folder_path, filename), "r") as f:
            lyrics = f.read()

        # append the data to the list
        data.append({"가수": artist, "제목": title, "가사": lyrics})

# create a pandas dataframe from the data
df = pd.DataFrame(data)

# save the dataframe to a csv file
df.to_csv("lyrics.csv", index=False)
